## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [14]:
# Установка Pykeen
!pip install pykeen

Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [15]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [Nations](https://paperswithcode.com/dataset/nations), предоставляющий информацию о связях между стран и их политических отношений. Этот небольшой датасет хорошо подходит для погружения в задачу предсказания отношений между сущностями.


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [16]:
from pykeen.datasets import Nations

Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [17]:
# Загрузка графового датасета Nations
nations_dataset = Nations()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [18]:
# Просмотр описания датасета
print(nations_dataset)

Nations(training_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/nations/train.txt", testing_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/nations/test.txt", validation_path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/nations/valid.txt")


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [19]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = nations_dataset.training.mapped_triples
print(training_data[:5])

tensor([[ 0,  3,  2],
        [ 0,  3,  3],
        [ 0,  3, 10],
        [ 0,  3, 13],
        [ 0,  4, 11]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [20]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - nations_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [21]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [22]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=nations_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.


Evaluating on cuda:0:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.11s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [23]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=2034683528, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(14, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(55, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=14, num_relations=55, create_inverse_triples=False, num_triples=1592, path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/nations/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7bfba87f4d90>, losses=[1.4655367987496513, 1.3687097345079695, 1.3480917726244246, 1.3597608293805803, 1.3459234067371912, 1.3577273743493217, 1.34379700251988, 1.2687709501811437, 1.2537677798952376, 1.2590818916048323, 1.1980326686586653, 1.2387174027306693, 1.2347616979054041, 1.2539423704147339, 1.1381591728755407, 1.1958

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [24]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [25]:
pack = predict_triples(model=pipeline_results.model, triples=nations_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [26]:
df.nlargest(n=10, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
65,4,egypt,18,exports3,11,uk,-8.263323
188,13,ussr,28,ngoorgs3,12,usa,-8.269464
90,7,israel,13,eemigrants,12,usa,-8.283918
86,6,indonesia,32,relbooktranslations,12,usa,-8.311117
126,10,poland,18,exports3,11,uk,-8.504192
83,6,indonesia,14,embassy,11,uk,-8.517539
10,0,brazil,42,reltreaties,12,usa,-8.573208
137,10,poland,42,reltreaties,11,uk,-8.581698
84,6,indonesia,23,militaryactions,2,china,-8.601527
99,7,israel,51,unweightedunvote,12,usa,-8.746500


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.